In [41]:
#!pip install lazypredict

# INFO Data

In [42]:
import numpy as np
import pandas as pd
import re
import json
import os

import seaborn as sns
import matplotlib.pyplot as plt

config_path = os.path.join('..', 'config.json')

with open(config_path, 'r') as config_file:
    config = json.load(config_file)

### 1. Data - info only

In [43]:
from sklearn.model_selection import train_test_split

data = None
data_d = None
data_t2 = None

X = None
Y = None

X_train = None
X_test = None
y_train = None
y_test = None

data_path = config['data_path']

input_path1 = os.path.join(data_path, 'analysis_to_python.dta')
data = pd.read_stata(input_path1, convert_categoricals=False)

# data subset
data_t2 = data[(data["treatment"]==6) & (data["info"]==1)] # train-test taken from treatment 6 and info only
# data_t2 = data_t2[["donation", "age", "educ", "income", "female", "children", "prev_donation_charity", "prev_donation_foreigner", "prev_volunteering", "prev_blooddonation"]]
data_t2 = data_t2[["donation", "age", "educ", "income", "female", "children"]]

data_t2 = data_t2.dropna() # 2 observations dropped

# X = data_t2[["age", "educ", "income", "female", "children", "prev_donation_charity", "prev_donation_foreigner", "prev_volunteering", "prev_blooddonation"]]
# y = data_t2["donation"]

X = data_t2[["age", "educ", "income", "female", "children"]]
y = data_t2["donation"]

X_train, X_test, y_train, y_test = train_test_split( X, y,  test_size=0.2, random_state = 100)

### 1.1 LazyClassifier

In [44]:
# import lazypredict
# from lazypredict.Supervised import LazyClassifier

# clf = LazyClassifier(verbose=0,ignore_warnings=True)
# models, predictions = clf.fit(X_train, X_test, y_train, y_test)

# print(models.to_markdown())

## 2. ML Methods

In [45]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

### 2.1 Random Forest

In [46]:
# from sklearn.ensemble import RandomForestClassifier

# # RF with 3 features and 100 trees
# rf3 = RandomForestClassifier(n_estimators=100, max_features=3, random_state=334)
# rfOpt = rf3.fit(X_train, y_train)

# # Prediction on the test set

# yChapRF = rfOpt.predict(X_test)

# # Confusion matrix

# tableRF = pd.crosstab(yChapRF, y_test)
# print(tableRF)

# # Prediction error on the test set
# print("Test accuracy - Random forest = %f" % (rfOpt.score(X_test, y_test)))

# ### Accuracy with random forest opt -quantitative = 0.798450

In [47]:
# importances_total = rf3.feature_importances_
# std_total = np.std([rf3.feature_importances_ for tree in rf3.estimators_], axis=0)

# feature_names_total = X.columns.values.tolist()

# forest_importances_total = pd.Series(importances_total, index=feature_names_total)

# plt.figure(figsize=(6,4))
# fig, ax = plt.subplots()
# forest_importances_total.plot.bar(yerr=std_total, ax=ax)
# ax.set_title("RF - Feature importances using MDI")
# ax.set_ylabel("Mean decrease in impurity")
# fig.tight_layout()

In [48]:
# %matplotlib inline

# (pd.Series(importances_total, index=feature_names_total)
#    .nlargest(20)
#    .plot(kind='barh'))

Hyperparameter tuning

In [49]:
# from scipy.stats import randint
# from sklearn.model_selection import RandomizedSearchCV

# rs_space={'max_depth':list(np.arange(10, 100, step=10)) + [None],
#               'n_estimators':np.arange(10, 500, step=50),
#               'max_features':randint(1,7),
#               'criterion':['gini','entropy'],
#               'min_samples_leaf':randint(1,4),
#               'min_samples_split':np.arange(2, 10, step=2)
#          }

# rf = RandomForestClassifier()
# rf_random = RandomizedSearchCV(rf, rs_space, n_iter=500, scoring='accuracy', n_jobs=-1, cv=3)
# model_random = rf_random.fit(X,y)


# print('Best hyperparameters are: '+str(model_random.best_params_))
# print('Best score is: '+str(model_random.best_score_))

# # Best hyperparameters are: {'criterion': 'gini', 'max_depth': 40, 'max_features': 1, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 60}
# # Best score is: 0.7477612726506532

In [50]:
# # RF with tuned parameters
# rf_opt = RandomForestClassifier(criterion='gini', max_depth=40, min_samples_leaf=3, min_samples_split=2, n_estimators=60, max_features=1, random_state=222)
# rf_opt_fit = rf_opt.fit(X_train, y_train)

# # Prediction on the test set
# yChapRF = None
# yChapRF = rf_opt_fit.predict(X_test)

# # Confusion matrix
# tableRF = None
# tableRF = pd.crosstab(yChapRF, y_test)
# print(tableRF)

# # Prediction error on the test set
# print("Test accuracy - Random forest opt= %f" % (rfOpt.score(X_test, y_test)))

# ### Accuracy with random forest opt = 0.764706

In [51]:
# print(tableRF.to_latex(index=True))

In [52]:
# # Predict Passive observer donations
# data_t4 = None
# data_t4_sub = None
# data_m = None

# data_t4 = data[data["treatment"]==4]
# data_t4_sub = data_t4[["age", "educ", "income", "female", "children", "prev_donation_charity", "prev_donation_foreigner", "prev_volunteering", "prev_blooddonation"]]
# data_t4_sub = data_t4_sub.fillna(data_t4_sub.mean()) # fill NAs with average (two rows)

# t4_pred = rf_opt_fit.predict(data_t4_sub)

# data_t4['rfpred_donation'] = t4_pred
# data_m = pd.merge(data, data_t4[['ID', 'rfpred_donation']], on='ID', how='left')

# data_m.to_stata('data_predicted.dta', version=118)

### 2.2 Gaussian NB

In [53]:
from sklearn.naive_bayes import GaussianNB

GNB = None

yChapL = None
table = None


GNB = GaussianNB().fit(X_train, y_train)

# custom threshold for binary predictions
def custom_predictG(X, threshold):
    probs = GNB.predict_proba(X)
    return (probs[:, 1] > threshold).astype(int)

# Prediction
yChapG = custom_predictG(X_test, 0.42)

tableG = pd.crosstab(yChapG, y_test)
print(tableG.to_markdown())
print(tableG.to_latex(index=True))

### Comment:

|   row_0 |   0.0 |   1.0 |
|--------:|------:|------:|
|       0 |    37 |    11 |
|       1 |    12 |    11 |
\begin{tabular}{lrr}
\toprule
donation &  0.0 &  1.0 \\
row\_0 &      &      \\
\midrule
0     &   37 &   11 \\
1     &   12 &   11 \\
\bottomrule
\end{tabular}



/var/folders/hq/srpr_0qj2xn8xc6w5cyc_rhh0000gn/T/ipykernel_38054/3853584826.py:21: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(tableG.to_latex(index=True))


### 2.3 K-Nearest Neighbors

In [54]:
from sklearn.neighbors import KNeighborsClassifier

KNN = None
KNN_CV = None
KNN_opt = None
KNN_opt_fit = None

yChapK = None
table = None

# Grid of penalty parameters
param = None
param = [{"n_neighbors": [1,2,3,4,5,6,7,8,9,10]}] # dictionary of the values of n_neighbors used for GridSearchCV


KNN_CV = GridSearchCV(KNeighborsClassifier(), param ,cv =5, n_jobs = -1)
KNN = KNN_CV.fit(X_train, y_train)

KNN.best_params_["n_neighbors"] # best n_neighbor is 8

# Computation of the loss and determining best parameter

print("Best learning rate = %f, Best parameter  = %s" %
      (1.-KNN.best_score_ , KNN.best_params_))

# Optimized model using the best parameter
KNN_opt = KNeighborsClassifier(n_neighbors=8)
KNN_opt_fit = KNN_opt.fit(X_train, y_train)

# custom threshold for binary predictions
def custom_predict(X, threshold):
    probs = KNN_opt.predict_proba(X)
    return (probs[:, 1] > threshold).astype(int)

# Prediction
yChapK = custom_predict(X_test, 0.37499)

table = pd.crosstab(yChapK, y_test)
print(table.to_markdown())

# Error on the test set
print("Test accuracy - Logistic regression  Lasso = %f" % (KNN_opt_fit.score(X_test, y_test)))
r2_score_KNN = KNN_opt_fit.score(X_test, y_test)




### Comment:

Best learning rate = 0.263910, Best parameter  = {'n_neighbors': 8}
|   row_0 |   0.0 |   1.0 |
|--------:|------:|------:|
|       0 |    33 |    11 |
|       1 |    16 |    11 |
Test accuracy - Logistic regression  Lasso = 0.661972


### 2.4 Linear Probability **Model**

In [55]:
linearPM_fit = None

yChapLPM = None
table = None


linearPM_fit = LinearRegression().fit(X_train, y_train)

# custom threshold for binary predictions
def custom_predictLPM(X, threshold):
    probs = linearPM_fit.predict(X)
    return (probs > threshold).astype(int)

# Prediction
yChapLPM = custom_predictLPM(X_test, 0.325)

tableLPM = pd.crosstab(yChapLPM, y_test)
print(tableLPM.to_markdown())
print(tableLPM.to_latex(index=True))


### Comment:

|   row_0 |   0.0 |   1.0 |
|--------:|------:|------:|
|       0 |    37 |    12 |
|       1 |    12 |    10 |
\begin{tabular}{lrr}
\toprule
donation &  0.0 &  1.0 \\
row\_0 &      &      \\
\midrule
0     &   37 &   12 \\
1     &   12 &   10 \\
\bottomrule
\end{tabular}



/var/folders/hq/srpr_0qj2xn8xc6w5cyc_rhh0000gn/T/ipykernel_38054/2572589795.py:19: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(tableLPM.to_latex(index=True))


### 2.5 Logistic Regression

#### 2.5.1 Lasso optimization

In [56]:
logitL = None
logitLasso = None
logitL_opt = None
logitL_opt_fit = None
yChapL = None
table = None

# Grid of penalty parameters
param = None
param = [{"C": [0.001,0.01,0.1,0.5,0.75,0.9,1,1.5,2,2.5,5]}] # dictionary of the values of C used for GridSearchCV

#COMMENT: We use the following codes to get logistic regression + l1 penalty (LASSO case)

logitL = GridSearchCV(LogisticRegression(penalty="l1", solver="liblinear"), param ,cv =5, n_jobs = -1)
logitLasso = logitL.fit(X_train, y_train)

logitLasso.best_params_["C"]

# Computation of the loss and determining best parameter

print("Best learning rate = %f, Best parameter  = %s" %
      (1.-logitLasso.best_score_ , logitLasso.best_params_))

# Optimized model using the best parameter
logitL_opt = LogisticRegression(C=0.001, penalty="l1", solver="liblinear")
logitL_opt_fit = logitL_opt.fit(X_train, y_train)

# custom threshold for binary predictions
def custom_predictL(X, threshold):
    probs = logitL_opt_fit.predict_proba(X)
    return (probs[:, 1] > threshold).astype(int)

# Prediction
yChapL = custom_predictL(X_test, 0.36)

tableLL = pd.crosstab(yChapL, y_test)
print(tableLL.to_markdown())

# Error on the test set
print("Test accuracy - Logistic regression  Lasso = %f" % (logitL_opt_fit.score(X_test, y_test)))
r2_score_lasso = logitL_opt_fit.score(X_test, y_test)

print(tableLL.to_latex(index=True))


### Comment:

Best learning rate = 0.257018, Best parameter  = {'C': 0.001}
|   row_0 |   0.0 |   1.0 |
|--------:|------:|------:|
|       0 |    33 |    16 |
|       1 |    16 |     6 |
Test accuracy - Logistic regression  Lasso = 0.690141
\begin{tabular}{lrr}
\toprule
donation &  0.0 &  1.0 \\
row\_0 &      &      \\
\midrule
0     &   33 &   16 \\
1     &   16 &    6 \\
\bottomrule
\end{tabular}



/var/folders/hq/srpr_0qj2xn8xc6w5cyc_rhh0000gn/T/ipykernel_38054/44237097.py:43: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(tableLL.to_latex(index=True))


#### 2.5.2 Ridge optimization

In [57]:
logitR = None
logitRidge = None
logitR_opt = None
logitR_opt_fit = None
yChapR = None
tableR = None

# Grid of penalty parameters
param = None
param = [{"C":[0.001,0.005,0.01,0.1,0.25,0.5,0.75,1,2.5,5,7,10,15,20]}]


logitR = GridSearchCV(LogisticRegression(penalty="l2", solver="liblinear"), param, cv = 5, n_jobs = -1)
logitRidge = logitR.fit(X_train, y_train)
logitRidge.best_params_["C"]

# Error computation
print("Best score = %f, Best parameter = %s" %
      (1. - logitRidge.best_score_, logitRidge.best_params_))

# Optimized model using the best parameter
logitR_opt = LogisticRegression(C=15, penalty="l2", solver="liblinear")
logitR_opt_fit = logitR_opt.fit(X_train, y_train)

# custom threshold for binary predictions
def custom_predictR(X, threshold):
    probs = logitR_opt_fit.predict_proba(X)
    return (probs[:, 1] > threshold).astype(int)

# Prediction
yChapR = custom_predictR(X_test, 0.32)

# confusion matrix
tableLR = pd.crosstab(yChapR, y_test)
print(tableLR.to_markdown())

# Error on the test set
print("Test accuracy - Logistic regression Ridge = %f" % (logitR_opt_fit.score(X_test, y_test)))
r2_score_ridge = logitR_opt_fit.score(X_test, y_test)

print(tableLR.to_latex(index=True))

### Comment: slightly better than Lasso, in all runs

Best score = 0.257018, Best parameter = {'C': 0.001}
|   row_0 |   0.0 |   1.0 |
|--------:|------:|------:|
|       0 |    37 |    12 |
|       1 |    12 |    10 |
Test accuracy - Logistic regression Ridge = 0.690141
\begin{tabular}{lrr}
\toprule
donation &  0.0 &  1.0 \\
row\_0 &      &      \\
\midrule
0     &   37 &   12 \\
1     &   12 &   10 \\
\bottomrule
\end{tabular}



/var/folders/hq/srpr_0qj2xn8xc6w5cyc_rhh0000gn/T/ipykernel_38054/3049714600.py:41: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(tableLR.to_latex(index=True))


## 3. Predictions

In [58]:
# Predict Passive observer donations
data_t46_info = None
data_t46_info_sub = None
data_m = None
data_m0 = None

data_t46_both = None
data_t46_both_sub = None

# subset with only info to predict on
data_t46_info = data[((data["treatment"]==4) | (data["treatment"]==6)) & (data["info"]==1)]
data_t46_info_sub = data_t46_info[["age", "educ", "income", "female", "children"]]
data_t46_info_sub = data_t46_info_sub.fillna(data_t46_info_sub.mean()) # fill NAs with average

# subset with both to predict on
data_t46_both = data[(data["treatment"]==4) | (data["treatment"]==6)]
data_t46_both_sub = data_t46_both[["age", "educ", "income", "female", "children"]]
data_t46_both_sub = data_t46_both_sub.fillna(data_t46_both_sub.mean()) # fill NAs with average

# prediction
info_Lpred = custom_predictL(data_t46_info_sub, 0.36)
info_Rpred = custom_predictR(data_t46_info_sub, 0.32)
info_LPMpred = custom_predictLPM(data_t46_info_sub, 0.325)
info_Gpred = custom_predictG(data_t46_info_sub, 0.42)

both_Lpred = custom_predictL(data_t46_both_sub, 0.36)
both_Rpred = custom_predictR(data_t46_both_sub, 0.32)
both_LPMpred = custom_predictLPM(data_t46_both_sub, 0.325)
both_Gpred = custom_predictG(data_t46_both_sub, 0.42)


data_t46_info['LLpred_donation'] = info_Lpred
data_t46_info['LRpred_donation'] = info_Rpred
data_t46_info['LPMpred_donation'] = info_LPMpred
data_t46_info['Gpred_donation'] = info_Gpred

data_t46_both['LLpred_donation_allwithinfo'] = both_Lpred
data_t46_both['LRpred_donation_allwithinfo'] = both_Rpred
data_t46_both['LPMpred_donation_allwithinfo'] = both_LPMpred
data_t46_both['Gpred_donation_allwithinfo'] = both_Gpred

# merging
data_m0 = pd.merge(data_t46_both, data_t46_info[['ID', 'LLpred_donation', 'LRpred_donation', 'LPMpred_donation', 'Gpred_donation']], on='ID', how='left') # first merging no-info with all-subset
data_m = pd.merge(data, data_m0[['ID', 'LLpred_donation', 'LRpred_donation', 'LPMpred_donation', 'Gpred_donation', 'LLpred_donation_allwithinfo', 'LRpred_donation_allwithinfo', 'LPMpred_donation_allwithinfo', 'Gpred_donation_allwithinfo']], on='ID', how='left') # second merging all-subset with whole data

output_path1 = os.path.join(data_path, 'data_predicted_info.dta')
data_m.to_stata(output_path1, version=118)

/var/folders/hq/srpr_0qj2xn8xc6w5cyc_rhh0000gn/T/ipykernel_38054/2398330045.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_t46_info['LLpred_donation'] = info_Lpred
/var/folders/hq/srpr_0qj2xn8xc6w5cyc_rhh0000gn/T/ipykernel_38054/2398330045.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_t46_info['LRpred_donation'] = info_Rpred
/var/folders/hq/srpr_0qj2xn8xc6w5cyc_rhh0000gn/T/ipykernel_38054/2398330045.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [59]:
%reset -f

# NO INFO Data

In [60]:
import numpy as np
import pandas as pd
import re
import json
import os

import seaborn as sns
import matplotlib.pyplot as plt

config_path = os.path.join('..', 'config.json')

with open(config_path, 'r') as config_file:
    config = json.load(config_file)

### 1. Data - no info only

In [61]:
from sklearn.model_selection import train_test_split

data = None
data_d = None
data_t2 = None

X = None
Y = None

X_train = None
X_test = None
y_train = None
y_test = None

data_path = config['data_path']

input_path1 = os.path.join(data_path, 'analysis_to_python.dta')
data = pd.read_stata(input_path1, convert_categoricals=False)


# data subset
data_t2 = data[(data["treatment"]==6) & (data["info"]==0)] # train-test taken from treatment 6 and no-info only
# data_t2 = data_t2[["donation", "age", "educ", "income", "female", "children", "prev_donation_charity", "prev_donation_foreigner", "prev_volunteering", "prev_blooddonation"]]
data_t2 = data_t2[["donation", "age", "educ", "income", "female", "children"]]

data_t2 = data_t2.dropna() # 2 observations dropped

# X = data_t2[["age", "educ", "income", "female", "children", "prev_donation_charity", "prev_donation_foreigner", "prev_volunteering", "prev_blooddonation"]]
# y = data_t2["donation"]

X = data_t2[["age", "educ", "income", "female", "children"]]
y = data_t2["donation"]

X_train, X_test, y_train, y_test = train_test_split( X, y,  test_size=0.2, random_state = 100)

## 2. ML Methods

In [62]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

### 2.1 Random Forest

In [63]:
# from sklearn.ensemble import RandomForestClassifier

# # RF with 3 features and 100 trees
# rf3 = RandomForestClassifier(n_estimators=100, max_features=3, random_state=334)
# rfOpt = rf3.fit(X_train, y_train)

# # Prediction on the test set

# yChapRF = rfOpt.predict(X_test)

# # Confusion matrix

# tableRF = pd.crosstab(yChapRF, y_test)
# print(tableRF)

# # Prediction error on the test set
# print("Test accuracy - Random forest = %f" % (rfOpt.score(X_test, y_test)))

# ### Accuracy with random forest opt -quantitative = 0.798450

In [64]:
# importances_total = rf3.feature_importances_
# std_total = np.std([rf3.feature_importances_ for tree in rf3.estimators_], axis=0)

# feature_names_total = X.columns.values.tolist()

# forest_importances_total = pd.Series(importances_total, index=feature_names_total)

# plt.figure(figsize=(6,4))
# fig, ax = plt.subplots()
# forest_importances_total.plot.bar(yerr=std_total, ax=ax)
# ax.set_title("RF - Feature importances using MDI")
# ax.set_ylabel("Mean decrease in impurity")
# fig.tight_layout()

In [65]:
# %matplotlib inline

# (pd.Series(importances_total, index=feature_names_total)
#    .nlargest(20)
#    .plot(kind='barh'))

Hyperparameter tuning

In [66]:
# from scipy.stats import randint
# from sklearn.model_selection import RandomizedSearchCV

# rs_space={'max_depth':list(np.arange(10, 100, step=10)) + [None],
#               'n_estimators':np.arange(10, 500, step=50),
#               'max_features':randint(1,7),
#               'criterion':['gini','entropy'],
#               'min_samples_leaf':randint(1,4),
#               'min_samples_split':np.arange(2, 10, step=2)
#          }

# rf = RandomForestClassifier()
# rf_random = RandomizedSearchCV(rf, rs_space, n_iter=500, scoring='accuracy', n_jobs=-1, cv=3)
# model_random = rf_random.fit(X,y)


# print('Best hyperparameters are: '+str(model_random.best_params_))
# print('Best score is: '+str(model_random.best_score_))

# # Best hyperparameters are: {'criterion': 'gini', 'max_depth': 40, 'max_features': 1, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 60}
# # Best score is: 0.7477612726506532

In [67]:
# # RF with tuned parameters
# rf_opt = RandomForestClassifier(criterion='gini', max_depth=40, min_samples_leaf=3, min_samples_split=2, n_estimators=60, max_features=1, random_state=222)
# rf_opt_fit = rf_opt.fit(X_train, y_train)

# # Prediction on the test set
# yChapRF = None
# yChapRF = rf_opt_fit.predict(X_test)

# # Confusion matrix
# tableRF = None
# tableRF = pd.crosstab(yChapRF, y_test)
# print(tableRF)

# # Prediction error on the test set
# print("Test accuracy - Random forest opt= %f" % (rfOpt.score(X_test, y_test)))

# ### Accuracy with random forest opt = 0.764706

In [68]:
# print(tableRF.to_latex(index=True))

In [69]:
# # Predict Passive observer donations
# data_t4 = None
# data_t4_sub = None
# data_m = None

# data_t4 = data[data["treatment"]==4]
# data_t4_sub = data_t4[["age", "educ", "income", "female", "children", "prev_donation_charity", "prev_donation_foreigner", "prev_volunteering", "prev_blooddonation"]]
# data_t4_sub = data_t4_sub.fillna(data_t4_sub.mean()) # fill NAs with average (two rows)

# t4_pred = rf_opt_fit.predict(data_t4_sub)

# data_t4['rfpred_donation'] = t4_pred
# data_m = pd.merge(data, data_t4[['ID', 'rfpred_donation']], on='ID', how='left')

# data_m.to_stata('data_predicted.dta', version=118)

### 2.2 Gaussian NB

In [70]:
# prompt: create a table showing the frequency of 'donation' values in 'X_test'

donation_counts = y_test.value_counts()
print(donation_counts.to_frame().reset_index().rename(columns={'index':'donation', 'donation':'count'}))


   donation  count
0       0.0     41
1       1.0     19


In [71]:
from sklearn.naive_bayes import GaussianNB

GNB = None

yChapL = None
table = None


GNB = GaussianNB().fit(X_train, y_train)

# custom threshold for binary predictions
def custom_predictG(X, threshold):
    probs = GNB.predict_proba(X)
    return (probs[:, 1] > threshold).astype(int)

# Prediction
yChapG = custom_predictG(X_test, 0.2386)

tableG = pd.crosstab(yChapG, y_test)
print(tableG.to_markdown())
print(tableG.to_latex(index=True))

### Comment:

|   row_0 |   0.0 |   1.0 |
|--------:|------:|------:|
|       0 |    32 |    13 |
|       1 |     9 |     6 |
\begin{tabular}{lrr}
\toprule
donation &  0.0 &  1.0 \\
row\_0 &      &      \\
\midrule
0     &   32 &   13 \\
1     &    9 &    6 \\
\bottomrule
\end{tabular}



/var/folders/hq/srpr_0qj2xn8xc6w5cyc_rhh0000gn/T/ipykernel_38054/546946962.py:21: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(tableG.to_latex(index=True))


### 2.3 K-Nearest Neighbors

In [72]:
from sklearn.neighbors import KNeighborsClassifier

KNN = None
KNN_CV = None
KNN_opt = None
KNN_opt_fit = None

yChapK = None
table = None

# Grid of penalty parameters
param = None
param = [{"n_neighbors": [1,2,3,4,5,6,7,8,9,10]}] # dictionary of the values of n_neighbors used for GridSearchCV


KNN_CV = GridSearchCV(KNeighborsClassifier(), param ,cv =5, n_jobs = -1)
KNN = KNN_CV.fit(X_train, y_train)

KNN.best_params_["n_neighbors"] # best n_neighbor is 8

# Computation of the loss and determining best parameter

print("Best learning rate = %f, Best parameter  = %s" %
      (1.-KNN.best_score_ , KNN.best_params_))

# Optimized model using the best parameter
KNN_opt = KNeighborsClassifier(n_neighbors=8)
KNN_opt_fit = KNN_opt.fit(X_train, y_train)

# custom threshold for binary predictions
def custom_predict(X, threshold):
    probs = KNN_opt.predict_proba(X)
    return (probs[:, 1] > threshold).astype(int)

# Prediction
yChapK = custom_predict(X_test, 0.37499)

tableKNN = pd.crosstab(yChapK, y_test)
print(tableKNN.to_markdown())

# Error on the test set
print("Test accuracy - Logistic regression  Lasso = %f" % (KNN_opt_fit.score(X_test, y_test)))
r2_score_KNN = KNN_opt_fit.score(X_test, y_test)


print(tableKNN.to_latex(index=True))


### Comment:

Best learning rate = 0.179876, Best parameter  = {'n_neighbors': 4}
|   row_0 |   0.0 |   1.0 |
|--------:|------:|------:|
|       0 |    35 |    12 |
|       1 |     6 |     7 |
Test accuracy - Logistic regression  Lasso = 0.683333
\begin{tabular}{lrr}
\toprule
donation &  0.0 &  1.0 \\
row\_0 &      &      \\
\midrule
0     &   35 &   12 \\
1     &    6 &    7 \\
\bottomrule
\end{tabular}



/var/folders/hq/srpr_0qj2xn8xc6w5cyc_rhh0000gn/T/ipykernel_38054/3898371262.py:46: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(tableKNN.to_latex(index=True))


### 2.4 Linear Probability **Model**

In [73]:
linearPM_fit = None

yChapLPM = None
table = None


linearPM_fit = LinearRegression().fit(X_train, y_train)

# custom threshold for binary predictions
def custom_predictLPM(X, threshold):
    probs = linearPM_fit.predict(X)
    return (probs > threshold).astype(int)

# Prediction
yChapLPM = custom_predictLPM(X_test, 0.2455)

tableLPM = pd.crosstab(yChapLPM, y_test)
print(tableLPM.to_markdown())
print(tableLPM.to_latex(index=True))


### Comment:

|   row_0 |   0.0 |   1.0 |
|--------:|------:|------:|
|       0 |    35 |    10 |
|       1 |     6 |     9 |
\begin{tabular}{lrr}
\toprule
donation &  0.0 &  1.0 \\
row\_0 &      &      \\
\midrule
0     &   35 &   10 \\
1     &    6 &    9 \\
\bottomrule
\end{tabular}



/var/folders/hq/srpr_0qj2xn8xc6w5cyc_rhh0000gn/T/ipykernel_38054/2325475714.py:19: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(tableLPM.to_latex(index=True))


### 2.5 Logistic Regression

#### 2.5.1 Lasso optimization

In [74]:
logitL = None
logitLasso = None
logitL_opt = None
logitL_opt_fit = None
yChapL = None
table = None

# Grid of penalty parameters
param = None
param = [{"C": [0.001,0.01,0.1,0.5,0.75,0.9,1,1.5,2,2.5,5]}] # dictionary of the values of C used for GridSearchCV

#COMMENT: We use the following codes to get logistic regression + l1 penalty (LASSO case)

logitL = GridSearchCV(LogisticRegression(penalty="l1", solver="liblinear"), param ,cv =5, n_jobs = -1)
logitLasso = logitL.fit(X_train, y_train)

logitLasso.best_params_["C"]

# Computation of the loss and determining best parameter

print("Best learning rate = %f, Best parameter  = %s" %
      (1.-logitLasso.best_score_ , logitLasso.best_params_))

# Optimized model using the best parameter
logitL_opt = LogisticRegression(C=0.001, penalty="l1", solver="liblinear")
logitL_opt_fit = logitL_opt.fit(X_train, y_train)

# custom threshold for binary predictions
def custom_predictL(X, threshold):
    probs = logitL_opt_fit.predict_proba(X)
    return (probs[:, 1] > threshold).astype(int)

# Prediction
yChapL = custom_predictL(X_test, 0.37)

tableLL = pd.crosstab(yChapL, y_test)
print(tableLL.to_markdown())

# Error on the test set
print("Test accuracy - Logistic regression  Lasso = %f" % (logitL_opt_fit.score(X_test, y_test)))
r2_score_lasso = logitL_opt_fit.score(X_test, y_test)


print(tableLL.to_latex(index=True))


### Comment:

Best learning rate = 0.179876, Best parameter  = {'C': 0.9}
|   row_0 |   0.0 |   1.0 |
|--------:|------:|------:|
|       0 |    38 |    17 |
|       1 |     3 |     2 |
Test accuracy - Logistic regression  Lasso = 0.683333
\begin{tabular}{lrr}
\toprule
donation &  0.0 &  1.0 \\
row\_0 &      &      \\
\midrule
0     &   38 &   17 \\
1     &    3 &    2 \\
\bottomrule
\end{tabular}



/var/folders/hq/srpr_0qj2xn8xc6w5cyc_rhh0000gn/T/ipykernel_38054/1791524234.py:44: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(tableLL.to_latex(index=True))


#### 2.5.2 Ridge optimization

In [75]:
logitR = None
logitRidge = None
logitR_opt = None
logitR_opt_fit = None
yChapR = None
tableR = None

# Grid of penalty parameters
param = None
param = [{"C":[0.001,0.005,0.01,0.1,0.25,0.5,0.75,1,2.5,5,7,10,15,20]}]


logitR = GridSearchCV(LogisticRegression(penalty="l2", solver="liblinear"), param, cv = 5, n_jobs = -1)
logitRidge = logitR.fit(X_train, y_train)
logitRidge.best_params_["C"]

# Error computation
print("Best score = %f, Best parameter = %s" %
      (1. - logitRidge.best_score_, logitRidge.best_params_))

# Optimized model using the best parameter
logitR_opt = LogisticRegression(C=15, penalty="l2", solver="liblinear")
logitR_opt_fit = logitR_opt.fit(X_train, y_train)

# custom threshold for binary predictions
def custom_predictR(X, threshold):
    probs = logitR_opt_fit.predict_proba(X)
    return (probs[:, 1] > threshold).astype(int)

# Prediction
yChapR = custom_predictR(X_test, 0.249)

# confusion matrix
tableLR = pd.crosstab(yChapR, y_test)
print(tableLR.to_markdown())

# Error on the test set
print("Test accuracy - Logistic regression Ridge = %f" % (logitR_opt_fit.score(X_test, y_test)))
r2_score_ridge = logitR_opt_fit.score(X_test, y_test)


print(tableLR.to_latex(index=True))

### Comment: slightly better than Lasso, in all runs

Best score = 0.184043, Best parameter = {'C': 0.001}
|   row_0 |   0.0 |   1.0 |
|--------:|------:|------:|
|       0 |    34 |    11 |
|       1 |     7 |     8 |
Test accuracy - Logistic regression Ridge = 0.683333
\begin{tabular}{lrr}
\toprule
donation &  0.0 &  1.0 \\
row\_0 &      &      \\
\midrule
0     &   34 &   11 \\
1     &    7 &    8 \\
\bottomrule
\end{tabular}



/var/folders/hq/srpr_0qj2xn8xc6w5cyc_rhh0000gn/T/ipykernel_38054/935477074.py:42: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(tableLR.to_latex(index=True))


## 3. Predictions

In [76]:
# Predict Passive observer donations
data_t46_info = None
data_t46_info_sub = None
data_m = None
data_m0 = None

data_t46_both = None
data_t46_both_sub = None

# subset with only no-info to predict on
data_t46_info = data[((data["treatment"]==4) | (data["treatment"]==6)) & (data["info"]==0)]
data_t46_info_sub = data_t46_info[["age", "educ", "income", "female", "children"]]
data_t46_info_sub = data_t46_info_sub.fillna(data_t46_info_sub.mean()) # fill NAs with average

# subset with both to predict on
data_t46_both = data[(data["treatment"]==4) | (data["treatment"]==6)]
data_t46_both_sub = data_t46_both[["age", "educ", "income", "female", "children"]]
data_t46_both_sub = data_t46_both_sub.fillna(data_t46_both_sub.mean()) # fill NAs with average

# prediction
info_Lpred = custom_predictL(data_t46_info_sub, 0.37)
info_Rpred = custom_predictR(data_t46_info_sub, 0.249)
info_LPMpred = custom_predictLPM(data_t46_info_sub, 0.2455)
info_Gpred = custom_predictG(data_t46_info_sub, 0.2386)

both_Lpred = custom_predictL(data_t46_both_sub, 0.37)
both_Rpred = custom_predictR(data_t46_both_sub, 0.249)
both_LPMpred = custom_predictLPM(data_t46_both_sub, 0.2455)
both_Gpred = custom_predictG(data_t46_both_sub, 0.2386)


data_t46_info['LLpred_donation'] = info_Lpred
data_t46_info['LRpred_donation'] = info_Rpred
data_t46_info['LPMpred_donation'] = info_LPMpred
data_t46_info['Gpred_donation'] = info_Gpred

data_t46_both['LLpred_donation_allwithnoinfo'] = both_Lpred
data_t46_both['LRpred_donation_allwithnoinfo'] = both_Rpred
data_t46_both['LPMpred_donation_allwithnoinfo'] = both_LPMpred
data_t46_both['Gpred_donation_allwithnoinfo'] = both_Gpred

# merging
data_m0 = pd.merge(data_t46_both, data_t46_info[['ID', 'LLpred_donation', 'LRpred_donation', 'LPMpred_donation', 'Gpred_donation']], on='ID', how='left') # first merging no-info with all-subset
data_m = pd.merge(data, data_m0[['ID', 'LLpred_donation', 'LRpred_donation', 'LPMpred_donation', 'Gpred_donation', 'LLpred_donation_allwithnoinfo', 'LRpred_donation_allwithnoinfo', 'LPMpred_donation_allwithnoinfo', 'Gpred_donation_allwithnoinfo']], on='ID', how='left') # second merging all-subset with whole data

output_path2 = os.path.join(data_path, 'data_predicted_noinfo.dta')
data_m.to_stata(output_path2, version=118)

/var/folders/hq/srpr_0qj2xn8xc6w5cyc_rhh0000gn/T/ipykernel_38054/4099974900.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_t46_info['LLpred_donation'] = info_Lpred
/var/folders/hq/srpr_0qj2xn8xc6w5cyc_rhh0000gn/T/ipykernel_38054/4099974900.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_t46_info['LRpred_donation'] = info_Rpred
/var/folders/hq/srpr_0qj2xn8xc6w5cyc_rhh0000gn/T/ipykernel_38054/4099974900.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [77]:
%reset -f

In [78]:
import pandas as pd
import json
import os

config_path = os.path.join('..', 'config.json')

with open(config_path, 'r') as config_file:
    config = json.load(config_file)


data_path = config['data_path']

input_path1 = os.path.join(data_path, 'data_predicted_info.dta')
input_path2 = os.path.join(data_path, 'data_predicted_noinfo.dta')


# merge no info and info prediction data sets
data_info = None
data_noinfo = None
data_final = None
data_final0 = None

data_info = pd.read_stata(input_path1, convert_categoricals=False)
data_noinfo = pd.read_stata(input_path2, convert_categoricals=False)

data_final0 = data_info.fillna(data_noinfo)
data_final = pd.merge(data_final0, data_noinfo[['ID', 'LLpred_donation_allwithnoinfo', 'LRpred_donation_allwithnoinfo', 'LPMpred_donation_allwithnoinfo', 'Gpred_donation_allwithnoinfo']], on='ID', how='left') # adding the four additional columns generated with no-info

output_path1 = os.path.join(data_path, 'analysis.dta')

data_final.to_stata(output_path1, version=118)